In [ ]:
%pip install transformers 
%pip install sklearn
%pip install datasets

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments,DataCollatorWithPadding
from datasets import load_dataset, load_metric,Dataset
import torch
import numpy as np
from utils import AssessData
from datasets import load_dataset
from utils import PrepareCorpus,AssessData
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset
import json
from re import template
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
parentdir = "/content/drive/MyDrive/ML/Datasets/20News"

train =parentdir+"/20news-bydate"+"/20news-bydate-train"
test = parentdir+"/20news-bydate"+ "/20news-bydate-test"

In [ ]:
id2labels = {'talk.politics.mideast': 0, 'rec.autos': 1, 'comp.sys.mac.hardware': 2, 'alt.atheism': 3, 'rec.sport.baseball': 4, 'comp.os.ms-windows.misc': 5, 'rec.sport.hockey': 6, 'sci.crypt': 7, 'sci.med': 8, 'talk.politics.misc': 9, 'rec.motorcycles': 10, 'comp.windows.x': 11, 'comp.graphics': 12, 'comp.sys.ibm.pc.hardware': 13, 'sci.electronics': 14, 'talk.politics.guns': 15, 'sci.space': 16, 'soc.religion.christian': 17, 'misc.forsale': 18, 'talk.religion.misc': 19}
id2classes = {'talk.politics.mideast': 0, 'rec.autos': 4, 'comp.sys.mac.hardware': 2, 'alt.atheism': 5, 'rec.sport.baseball': 1, 'comp.os.ms-windows.misc': 2, 'rec.sport.hockey': 1, 'sci.crypt': 3, 'sci.med': 3, 'talk.politics.misc': 0, 'rec.motorcycles': 4, 'comp.windows.x': 2, 'comp.graphics': 2, 'comp.sys.ibm.pc.hardware': 2, 'sci.electronics': 3, 'talk.politics.guns': 0, 'sci.space': 3, 'soc.religion.christian': 5, 'misc.forsale': 6, 'talk.religion.misc': 5}

max_input_length = 300
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

number = range(50,150,50)
_per_segment = range(200,400,100)

overlap = {"side":"both", "number": 50}

f1_score = load_metric("f1"); precision = load_metric("precision"); recall = load_metric("recall")

def tokenize(batch):
    return bert_tokenizer(batch['text'], truncation=True, max_length=max_input_length,padding = True)

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return {"f1" :f1_score.compute(predictions = predictions, references = label, average = 'weighted'),
            "precision" : precision.compute(predictions = predictions, references = label, average = 'weighted'),
            "recall": recall.compute(predictions = predictions, references = label, average = 'weighted')}

def hp_space(trial):
  return {"per_device_train_batch_size": trial.suggest_discrete_uniform("per_device_train_batch_size", 8,32,8)
        ,"learning_rate": trial.suggest_float("learning_rate", 0.00001,0.00005, log = True)
        ,"num_train_epochs": trial.suggest_int("num_train_epochs",1,10)}

def bert_init():
  return BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 7)

def id_2_labels(x,adict):
  """converts labels/classes into a number using a dictionary"""
  return adict[x]

def _indexing(alist):
  id = 0; indexed = {}
  for x in alist:
    for y in x:
      id +=1
      indexed[id] = y
  return indexed
  

In [ ]:
news_group_train = PrepareCorpus(train)
news_group_test = PrepareCorpus(test)

In [ ]:
news_group_test = news_group_test._prep()       #Keys are labels, values are texts
news_group_train = news_group_train._prep()

SUBCATEGORY MAPPING

In [ ]:
#select only politics
id_politics = {'talk.politics.misc': 0, 'talk.politics.mideast': 1, 'talk.politics.guns': 2}
id_religion = {'alt.atheism': 0, 'soc.religion.christian' : 1, 'talk.religion.misc': 2}
id_sport = {'rec.sport.baseball': 0, 'rec.sport.hockey': 1}
id_autos = {'rec.motorcycles': 0, 'rec.autos': 1,}
id_sci = {'sci.crypt': 0, 'sci.med': 1, 'sci.space': 2}
id_comp = {'comp.sys.mac.hardware': 0,  'comp.os.ms-windows.misc': 1, 'comp.windows.x': 2, 'comp.graphics': 3, 'comp.sys.ibm.pc.hardware': 4, 'sci.electronics': 5}
id_sale = {'misc.forsale': 0}

In [ ]:
#Selecting id_religion

temp_train = []; temp_test = []
temp_news_group_train = {}; temp_news_group_test = {}

for item in id_religion.keys():#change id as required 
  temp_news_group_train[item] = news_group_train.get(item)
  temp_news_group_test[item] = news_group_test.get(item)

temp_train = _indexing(temp_news_group_train.values())
temp_test =  _indexing(temp_news_group_test.values())

temp_train_corpus = AssessData(temp_train, temp_news_group_train)
temp_test_corpus = AssessData(temp_test, temp_news_group_test)

In [ ]:
type(temp_train_corpus)

utils.AssessData

In [ ]:
temp_train_index_to_label = temp_train_corpus._index_to_label() 
temp_test_index_to_label = temp_test_corpus._index_to_label()

In [ ]:
set(temp_train_index_to_label.values())

{'alt.atheism', 'soc.religion.christian', 'talk.religion.misc'}

In [ ]:
temp_train_index_to_label = {i:id_2_labels(x, id_religion) for i,x in temp_train_index_to_label.items()} #change id as required
temp_test_index_to_label = {i:id_2_labels(x, id_religion) for i,x in temp_test_index_to_label.items()}

In [ ]:
temp_train_set = temp_train_corpus._chunk(200, overlap=overlap)
temp_train_index, temp_train_text = zip(*temp_train_set)

temp_test_set = temp_test_corpus._chunk(200, overlap=overlap)
test_index, test_text = zip(*temp_test_set)

In [ ]:
train_index = {a:x-1 for a,x in enumerate(temp_train_index)}
train_text = {a:x for a,x in enumerate(temp_train_text)}

In [ ]:
test_index = {a:x-1 for a,x in enumerate(test_index)}
test_text = {a:x for a,x in enumerate(test_text)}

In [ ]:
train_chunked_label = {i:id_2_labels(x,temp_train_index_to_label) for i,x in train_index.items()}
test_chunked_label = {i:id_2_labels(x,temp_test_index_to_label) for i,x in test_index.items()}

In [ ]:
train_chunked_label

In [ ]:
rel_train_set = {"train" : {"text": train_text.values(), 'doc_id': train_index.values(), 'labels': train_chunked_label.values()}}
rel_test_set =  {"test" : {"text": test_text.values(), 'doc_id': test_index.values(), 'labels': test_chunked_label.values()}}

In [ ]:
import pandas as pd

In [ ]:
a = pd.DataFrame(rel_train_set['train'])
b = pd.DataFrame(rel_test_set['test'])

In [ ]:
a

,text,doc_id,labels
0,sgiblab!adagio.panasonic.com!nntp-server.calt...,0,0
1,h.edu> <1p88fi$4vv@fido.asd.sgi.com> \n <1p9b...,0,0
2,"ogy, Pasadena\n NNTP-Posting-Host: punisher.ca...",0,0
3,"pproximations to it. Once again, a natural or...",0,0
4,"te getting what you mean by this, but I think ...",0,0
...,...,...,...
18671,you have read too much into what the media ha...,1455,2
18672,ave denied any\n >> further ties with this cul...,1455,2
18673,condemn and \n >make this to a show case.\n \n...,1455,2
18674,capable of letting\n >their small children bu...,1455,2


In [ ]:
a['text'] = a['text'].astype(str)
a['doc_id'] = a['doc_id'].astype(int)
a['labels'] = a['labels'].astype(int)

b['text'] = a['text'].astype(str)
b['doc_id'] = a['doc_id'].astype(int)
b['labels'] = a['labels'].astype(int)

In [ ]:
rel_train_set = Dataset.from_pandas(a)
rel_test_set = Dataset.from_pandas(b)

In [ ]:
rel_train_set = rel_train_set.map(tokenize)
rel_test_set = rel_test_set.map(tokenize)

rel_train_set = rel_train_set.shuffle(seed = 24)
rel_test_set = rel_test_set.shuffle(seed = 24)

  0%|          | 0/18676 [00:00<?, ?ex/s]

  0%|          | 0/13179 [00:00<?, ?ex/s]

In [ ]:
rel_test_set

Dataset({
    features: ['text', 'doc_id', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 13179
})

In [ ]:
fold = KFold(n_splits = 5)
this_set = fold.split(rel_train_set['text']) #change

frac_fold_train = []; frac_fold_eval = []

for i,j in this_set:
  frac_fold_train.append(rel_train_set.select(i))
  frac_fold_eval.append(rel_train_set.select(j))

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer, max_length= 300)

bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = len(set(a['labels'])))

#default training arguments 
training_args = TrainingArguments(output_dir="./frac_results", learning_rate=2e-5, per_device_train_batch_size=16, per_device_eval_batch_size=16, num_train_epochs=5,weight_decay=0.01)
results = []

#fine-tune
trainer = Trainer(model = bert_model, 
                args = training_args,
                train_dataset= frac_fold_train[0],
                eval_dataset= frac_fold_eval[0],
                tokenizer= bert_tokenizer,
                data_collator=data_collator,
                compute_metrics= compute_metrics)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss
500,0.870300


Saving model checkpoint to ./frac_results/checkpoint-500
Configuration saved in ./frac_results/checkpoint-500/config.json
Model weights saved in ./frac_results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./frac_results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./frac_results/checkpoint-500/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2340: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, doc_id. If text, doc_id are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3736
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2340: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "


Trainer is attempting to log a value of "{'f1': 0.8120173644201918}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8162082854626251}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8137044967880086}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'epoch': 5.0,
 'eval_f1': {'f1': 0.8120173644201918},
 'eval_loss': 0.7616076469421387,
 'eval_precision': {'precision': 0.8162082854626251},
 'eval_recall': {'recall': 0.8137044967880086},
 'eval_runtime': 11.9859,
 'eval_samples_per_second': 311.699,
 'eval_steps_per_second': 19.523}

In [ ]:
answer = trainer.predict(rel_test_set)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, doc_id. If text, doc_id are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 13179
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2340: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
answer.predictions

array([[-2.0730433,  4.6317234, -2.9197907],
       [ 5.795626 , -2.8040988, -3.19997  ],
       [ 5.5562863, -2.8027637, -3.1799107],
       ...,
       [-2.919395 ,  5.817092 , -3.1869204],
       [-2.1953456,  5.10492  , -3.3028946],
       [ 5.919046 , -3.1173358, -3.0202005]], dtype=float32)

In [ ]:
predictions = np.argmax(answer.predictions, axis = 0)
predictions

array([5674, 5183, 6971])

In [ ]:
answer.label_ids

array([1, 0, 0, ..., 1, 1, 0])

In [ ]:
c = pd.DataFrame(rel_test_set['labels'], columns = ["orig"])
c['labels'] = predictions

In [ ]:
d = c.loc[c["labels"] != c["orig"]]

In [ ]:
test_index[128]

10

In [ ]:
def flip_dict(adict: dict):
  return {x:i for i,x in adict.items()}

In [ ]:
errors = zip(d.index, d.labels) #investigating errors
orig_docs_index = []
error_labels = []
true_labels = []

for a,b in errors:
  
  real_index = test_index[a] #retrieves real index before chunking
  orig_docs_index.append(real_index) 

  true_labels.append(flip_dict(id_religion)[b])
  error_labels.append(flip_dict(id_religion)[temp_test_index_to_label[real_index]]) #retrieves word labels


In [ ]:

for a,b in zip(true_labels, error_labels):
  print(a,b)

#proceed by examining sentence structure later

In [ ]:
rel_test_set['labels']